<a href="https://colab.research.google.com/github/AfshinRezakhani/Thesis1/blob/main/GAN_GRU_d6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Reshape, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# 1. بارگذاری دیتاست d6
file_path = "/content/d6.csv"
df = pd.read_csv(file_path)

# 2. پردازش داده‌ها
target_column = "F"
features = df.drop(columns=[target_column])
target = df[target_column]

# بررسی توزیع کلاس‌ها
print("🔹 توزیع کلاس‌های اصلی در داده‌ها:", np.unique(target, return_counts=True))

# 3. نرمال‌سازی داده‌ها
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# 4. تقسیم داده‌ها به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# 5. تغییر شکل داده‌ها برای ورودی GRU
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# 6. تعریف Generator بهبودیافته
def build_generator():
    model = Sequential([
        Input(shape=(1, X_train.shape[2])),
        GRU(512, return_sequences=True),
        BatchNormalization(),
        Dropout(0.3),
        GRU(256, return_sequences=True),
        BatchNormalization(),
        Dropout(0.3),
        GRU(128, return_sequences=False),
        Dense(X_train.shape[2]),
        Reshape((1, X_train.shape[2]))
    ])
    return model

# 7. تعریف Discriminator بهبودیافته
def build_discriminator():
    model = Sequential([
        Input(shape=(1, X_train.shape[2])),
        GRU(512, return_sequences=True),
        Dropout(0.4),
        BatchNormalization(),
        GRU(256, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    return model

# 8. تعریف مدل GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    return model

# 9. کامپایل مدل‌ها
discriminator = build_discriminator()
discriminator.compile(loss="binary_crossentropy", optimizer=Adam(0.00005, 0.5), metrics=["accuracy"])

generator = build_generator()
gan = build_gan(generator, discriminator)
gan.compile(loss="binary_crossentropy", optimizer=Adam(0.00005, 0.5))

# 10. آموزش مدل GAN
epochs = 3000
batch_size = 512
half_batch = batch_size // 2

for epoch in range(epochs):
    # ایجاد داده‌های جعلی
    noise = np.random.normal(0, 1, (half_batch, 1, X_train.shape[2]))
    generated_data = generator.predict(noise, verbose=0)

    # انتخاب داده‌های واقعی تصادفی
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    real_data = X_train[idx]

    # برچسب‌های واقعی و جعلی (Label Smoothing)
    real_labels = np.random.uniform(0.9, 1.0, (half_batch, 1))
    fake_labels = np.random.uniform(0.0, 0.1, (half_batch, 1))

    # ترکیب داده‌های واقعی و جعلی
    X_combined = np.concatenate((real_data, generated_data))
    y_combined = np.concatenate((real_labels, fake_labels))

    # آموزش Discriminator در هر 2 epoch
    if epoch % 2 == 0:
        d_loss = discriminator.train_on_batch(X_combined, y_combined)

    # آموزش Generator برای فریب دادن Discriminator
    noise = np.random.normal(0, 1, (batch_size, 1, X_train.shape[2]))
    y_mislabeled = np.ones((batch_size, 1))

    g_loss = gan.train_on_batch(noise, y_mislabeled)

    d_loss = float(d_loss[0]) if isinstance(d_loss, list) else float(d_loss)
    g_loss = float(g_loss[0]) if isinstance(g_loss, list) else float(g_loss)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Discriminator Loss: {d_loss:.4f}, Generator Loss: {g_loss:.4f}")

# 11. تولید داده‌های جدید برای کلاس اقلیت
minority_class = 0 if np.bincount(y_train)[0] < np.bincount(y_train)[1] else 1
num_samples_to_generate = abs(np.bincount(y_train)[0] - np.bincount(y_train)[1])

noise = np.random.normal(0, 1, (num_samples_to_generate, 1, X_train.shape[2]))
generated_samples = generator.predict(noise, verbose=0)

# ایجاد لیبل‌های جدید برای داده‌های تولید شده
synthetic_labels = np.full((num_samples_to_generate,), minority_class)

# 12. ترکیب داده‌های واقعی و داده‌های تولیدشده برای متعادل‌سازی کلاس‌ها
X_balanced = np.vstack((features_scaled, generated_samples.squeeze()))
y_balanced = np.concatenate((target, synthetic_labels))

# 13. ذخیره دیتاست بالانس‌شده
df_balanced = pd.DataFrame(X_balanced, columns=df.columns[:-1])
df_balanced[target_column] = y_balanced

balanced_file_path = "/content/d6_balanced_withGRU.csv"
df_balanced.to_csv(balanced_file_path, index=False)
print(f"✅ Balanced dataset saved as {balanced_file_path}")

# 14. بررسی توزیع کلاس‌ها در پیش‌بینی مدل
y_pred_probs = discriminator.predict(X_test, batch_size=512, verbose=1)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

print("🔹 توزیع کلاس‌ها در پیش‌بینی مدل:", np.unique(y_pred, return_counts=True))

# 15. محاسبه Accuracy و F1-Score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ F1-Score: {f1:.4f}")


🔹 توزیع کلاس‌های اصلی در داده‌ها: (array([0, 1]), array([2978, 7022]))


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0: Discriminator Loss: 0.6914, Generator Loss: 0.6932
Epoch 100: Discriminator Loss: 0.6919, Generator Loss: 0.6795
Epoch 200: Discriminator Loss: 0.6933, Generator Loss: 0.6678
Epoch 300: Discriminator Loss: 0.6970, Generator Loss: 0.6572
Epoch 400: Discriminator Loss: 0.7031, Generator Loss: 0.6475
